In [2]:
import pysmu
import numpy as np
import matplotlib.pyplot as plt

# Getting started

My google doc on these devices:

https://docs.google.com/document/d/1fo8Q-7d8k15Rejed2K7R8EPeDC1ZAO8q1dyLakdqkw4/edit

Good news is that there are conda packages! 

There are some basic examples:

https://github.com/analogdevicesinc/libsmu/tree/master/bindings/python/examples

But very little documentation of the python API...seems like the documentation is the source:

https://github.com/analogdevicesinc/libsmu/blob/master/bindings/python/pysmu/libsmu.pyx

Let's start by at least connecting to it.

In [5]:
session = pysmu.Session()

In [9]:
print(session.devices)
print(len(session.devices))
print(session.devices[0])

(<pysmu.libsmu.SessionDevice object at 0x118e439b0>,)
1
serial 2031205051485435313330323030323 : fw 2.17 : hw F


In [10]:
dev = session.devices[0]

In [44]:
def show(obj):
    for n in [s for s in dir(obj) if "__" not in s]:
        print(n)

In [45]:
show(dev)

calibration
channels
ctrl_transfer
default_rate
flash_firmware
flush
fwver
get_samples
hwver
ignore_dataflow
overcurrent
read
samba_mode
sample_rate
serial
set_adc_mux
set_led
write
write_calibration


In [46]:
print(dev.channels.keys())

odict_keys(['A', 'B'])


In [47]:
chA = dev.channels["A"]

In [49]:
show(chA)

arbitrary
constant
flush
get_samples
mode
read
sawtooth
signal
sine
square
stairstep
triangle
write


OK, we're getting there. There is zero documentation of the class unfortunately, `help(chA.mode)` is useless...

Maybe I dont't understand docstrings? There is something in the code itself: 

https://github.com/analogdevicesinc/libsmu/blob/master/bindings/python/pysmu/libsmu.pyx

Line 37: SIMV is "source current, measure voltage". But I find it still a bit fuzzy.

OK, the autogenerated c documenation also seems useless for actually explaining how to use this thing. I guess it will mostly be reverse engineering.

Maybe this file will contain useful examples to dig out:

https://github.com/analogdevicesinc/libsmu/blob/master/bindings/python/bin/pysmu

> This script provides similar support to the smu command-line utility mostly to
provide examples for how to leverage the pysmu bindings

Sounds like a good start :)

Well, was a bit useful, but I'm still not further. Let's try a focussed approach: googling to try to achieve my task of reading out the resistance of a PT1000 sensor. 

# Goal: PT1000 sensor readout

## Googling

There is an ohmmeter app that is written in python:

https://wiki.analog.com/university/tools/m1k/alice/ohmmeter-users-guide

But there does not seem to be any source code.

There is this wiki page:

https://wiki.analog.com/university/tools/python-tutorial/table-of-contents

but it seems to be horrendously outdated, based on an older python 2.7 version of the library.

OK, maybe a breakthrough: the python source of their "Alice" desktop software is available on github!

https://github.com/analogdevicesinc/alice/tree/Version-1.3

I think I'll dig into this code:

https://github.com/analogdevicesinc/alice/blob/Version-1.3/ohm-meter-vdiv-1.3.pyw
